In [1]:
import numpy as np
from a2gmeasurements import GimbalGremsyH16

gh16 = GimbalGremsyH16()

X1 = gh16.speed_time_azimuth_table[:, 0:2]
Y1 = np.rad2deg(gh16.speed_time_azimuth_table[:, -1])

X2 = gh16.speed_time_elevation_table[:, 0:2]
Y2 = np.rad2deg(gh16.speed_time_elevation_table[:, -1])

c:\Users\jvjulian\MyInstalledPrograms\envs\gimbalRS2\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


[DEBUG]: POSITIVE SPEEDS (LEFT), AZIMUTH, R^2 Score Linear Reg:  0.9828922068260324
[DEBUG]: NEGATIVE SPEEDS (RIGHT), AZIMUTH, R^2 Score Linear Reg:  0.970834177125614
[DEBUG]: NEGATIVE SPEEDS (DOWN), ELEVATION, R^2 Score Linear Reg:  0.8124531573621369
[DEBUG]: POSITIVE SPEEDS (UP), ELEVATION, R^2 Score Linear Reg:  0.8124531573621369


Splitting the dataset

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_temp, Y_train, Y_temp = train_test_split(X1, Y1, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

Polynomial Fitting

In [3]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

degree = 2

# Create polynomial features
poly = PolynomialFeatures(degree=degree)
X_poly_train = poly.fit_transform(X_train_scaled)
X_poly_val = poly.transform(X_val_scaled)
X_poly_test = poly.transform(X_test_scaled)

# Train the polynomial regression model
model = LinearRegression()
model.fit(X_poly_train, Y_train)

Y_train_pred = model.predict(X_poly_train)
mse = mean_squared_error(Y_train, Y_train_pred)
print(f"Mean Squared Error (Train): {mse}")

# Predict on the validation set
Y_val_pred = model.predict(X_poly_val)

# Calculate Mean Squared Error (MSE) on the validation set
mse = mean_squared_error(Y_val, Y_val_pred)
print(f"Mean Squared Error (Validation): {mse}")

# Predict on the test set
Y_test_pred = model.predict(X_poly_test)

# Calculate Mean Squared Error (MSE) on the test set
mse_test = mean_squared_error(Y_test, Y_test_pred)
print(f"Mean Squared Error (Test): {mse_test}")

Mean Squared Error (Train): 4.468206414612399
Mean Squared Error (Validation): 11.687336702316577
Mean Squared Error (Test): 11.458766085597468


Random Forest

In [37]:
from sklearn.ensemble import RandomForestRegressor

# Create a Random Forest regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(X_train_scaled, Y_train)

# Predict on the training set
Y_train_pred = rf_model.predict(X_train_scaled)

# Calculate Mean Squared Error (MSE) on the validation set
mse_train = mean_squared_error(Y_train, Y_train_pred)
print(f"Mean Squared Error (Training): {mse_train}")

# Predict on the validation set
Y_val_pred = rf_model.predict(X_val_scaled)

# Calculate Mean Squared Error (MSE) on the validation set
mse_val = mean_squared_error(Y_val, Y_val_pred)
print(f"Mean Squared Error (Validation): {mse_val}")

# Predict on the test set
Y_test_pred = rf_model.predict(X_test_scaled)

# Calculate Mean Squared Error (MSE) on the test set
mse_test = mean_squared_error(Y_test, Y_test_pred)
print(f"Mean Squared Error (Test): {mse_test}")

Mean Squared Error (Training): 5.486971891954302
Mean Squared Error (Validation): 17.88580995084673
Mean Squared Error (Test): 36.2932480238418


Gaussian Process

In [4]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.metrics import mean_squared_error

# Define the kernel for Gaussian Process Regressor
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-3))

# Create the Gaussian Process regression model
gp_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)

# Fit the model to the training data
gp_model.fit(X_train, Y_train)

# Predict on the validation set
Y_train_pred, _ = gp_model.predict(X_train, return_std=True)

# Calculate Mean Squared Error (MSE) on the validation set
mse_train = mean_squared_error(Y_train, Y_train_pred)
print(f"Mean Squared Error (Training): {mse_train}")

# Predict on the validation set
Y_val_pred, _ = gp_model.predict(X_val, return_std=True)

# Calculate Mean Squared Error (MSE) on the validation set
mse_val = mean_squared_error(Y_val, Y_val_pred)
print(f"Mean Squared Error (Validation): {mse_val}")

# Predict on the test set
Y_test_pred, _ = gp_model.predict(X_test, return_std=True)

# Calculate Mean Squared Error (MSE) on the test set
mse_test = mean_squared_error(Y_test, Y_test_pred)
print(f"Mean Squared Error (Test): {mse_test}")

Mean Squared Error (Training): 5.747646345030014e-09
Mean Squared Error (Validation): 7.372210776628667
Mean Squared Error (Test): 5.192961258664729


In [5]:
# Extract the hyperparameters
kernel_params = gp_model.kernel_.get_params()

print("Hyperparameters of the Gaussian Process Regressor:")
for key, value in kernel_params.items():
    print(f"{key}: {value}")

Hyperparameters of the Gaussian Process Regressor:
k1: 36.2**2 * RBF(length_scale=1.78)
k2: WhiteKernel(noise_level=0.00011)
k1__k1: 36.2**2
k1__k2: RBF(length_scale=1.78)
k1__k1__constant_value: 1309.2141753770754
k1__k1__constant_value_bounds: (1e-05, 100000.0)
k1__k2__length_scale: 1.7772929857424073
k1__k2__length_scale_bounds: (0.01, 100.0)
k2__noise_level: 0.0001101246710652555
k2__noise_level_bounds: (1e-10, 0.001)


In [8]:
import time 

start_time = time.time()
# Desired yaw angle for which you want to find speed and time
desired_yaw = 70.0  # Replace with the desired yaw angle

# Define a function to find the corresponding X values for the desired Y
def find_feature_values_for_yaw(model, desired_yaw):
    # Define a function to minimize (difference between predicted and desired Y)
    def objective_function(x):
        x = np.atleast_2d(x)
        return np.abs(model.predict(x) - desired_yaw)

    # Initialize with a guess for speed and time
    initial_guess = np.array([2.0, 3.0])

    # Use an optimization method to find the feature values that result in the desired yaw angle
    from scipy.optimize import minimize
    result = minimize(objective_function, initial_guess, method='Nelder-Mead')

    if result.success:
        return result.x
    else:
        raise Exception("Optimization did not converge.")

# Find the corresponding feature values (speed and time) for the desired yaw angle
corresponding_feature_values = find_feature_values_for_yaw(gp_model, desired_yaw)
total_time = time.time() - start_time

print(f"Total time: {total_time}")
print(f"Desired Yaw Angle: {desired_yaw}")
print(f"Corresponding Feature Values (Speed, Time): {corresponding_feature_values}")

Total time: 0.03317594528198242
Desired Yaw Angle: 70.0
Corresponding Feature Values (Speed, Time): [15.66756407  8.95242118]


Neural Network

In [45]:
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from keras_tuner import RandomSearch, GridSearch
import keras_tuner as kt

# Define a function to create a Keras model with variable hyperparameters
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=X_train.shape[1]))
    
    # Define the number of hidden layers and neurons based on hyperparameters
    for _ in range(hp.Int('num_hidden_layers', min_value=2, max_value=4, step=1)):
        model.add(keras.layers.Dense(units=hp.Int('units', min_value=50, max_value=150), activation='relu'))
    
    model.add(keras.layers.Dense(1))  # Output layer
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Initialize the KerasTuner
tuner = RandomSearch(
    hypermodel=build_model,
    objective=kt.Objective('val_mean_squared_error', direction='max'),
    max_trials=9
)

# Perform hyperparameter tuning
tuner.search(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100)

# Get the best hyperparameters and model
#best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
#best_model = tuner.hypermodel.build(best_hps)

#best_model = build_model(tuner.get_best_hyperparameters(1)[0])
#best_model.fit(X1, Y1, epochs=50)

best_model = tuner.get_best_models()[0]

# Train the best model on the entire training dataset
best_model.fit(X_train, Y_train, epochs=100, validation_data=(X_val, Y_val))

Reloading Tuner from .\untitled_project\tuner0.json
Epoch 1/100
1/1 [==============================] - 1s 706ms/step - loss: 3246.4941 - val_loss: 4408.4385
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 3232.0554 - val_loss: 4383.9658
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 3213.2395 - val_loss: 4355.7725
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 3191.5891 - val_loss: 4325.0132
Epoch 5/100
1/1 [==============================] - 0s 40ms/step - loss: 3167.9819 - val_loss: 4292.4648
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 3143.0112 - val_loss: 4258.6885
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 3117.1121 - val_loss: 4224.1016
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 3090.5869 - val_loss: 4189.0244
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 3063.6794 - val_loss: 4153.7588
Epoch 10/10

In [46]:
# Evaluate the best model on the test set
Y_train_pred = best_model.predict(X_train_scaled)
mse_train = mean_squared_error(Y_train, Y_train_pred)
print(f"Mean Squared Error (Training): {mse_train}")

# Evaluate the best model on the test set
Y_val_pred = best_model.predict(X_val_scaled)
mse_val = mean_squared_error(Y_val, Y_val_pred)
print(f"Mean Squared Error (Validation): {mse_val}")

# Evaluate the best model on the test set
Y_test_pred = best_model.predict(X_test_scaled)
mse_test = mean_squared_error(Y_test, Y_test_pred)
print(f"Mean Squared Error (Test): {mse_test}")

1/1 [==============================] - 0s 59ms/step
Mean Squared Error (Training): 2599.3557455288774
1/1 [==============================] - 0s 18ms/step
Mean Squared Error (Validation): 3554.5082491659914
1/1 [==============================] - 0s 19ms/step
Mean Squared Error (Test): 2947.172205682591


In [6]:
import plotly.express as px
import plotly.graph_objects as go

# Create scatter traces for each dataset with distinct marker attributes
trace1 = go.Scatter3d(x=X1[:, 0], y=X1[:, 1], z=Y_train, mode='markers', name='Data', marker=dict(size=5, symbol='circle', color='blue'))
trace2 = go.Scatter3d(x=X1[:, 0], y=X1[:, 1], z=Y_train_pred, mode='markers', name='Model', marker=dict(size=3, symbol='square', color='red'))

# Create a layout for the 3D figure
layout = go.Layout(scene=dict(xaxis_title='Speed', yaxis_title='Time', zaxis_title='Yaw'), title='Gremsy Gimbal Model')

# Create a 3D figure and add the scatter traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Show the 3D figure
fig.show()